In [6]:
def f(a, b):
    return 2 / ((1/a)+(1/b))


d = [
    (0.5, 2.5),
    (0.5, 2.5),
    (1, 1.67),
    (0.5, 2.5),
    (0.5, 2.5),
    (1, 1.67),
    (1, 1.67),
    (1, 5)
]

for a, b in d:
    print(f(a, b))

0.8333333333333334
0.8333333333333334
1.250936329588015
0.8333333333333334
0.8333333333333334
1.250936329588015
1.250936329588015
1.6666666666666667


In [9]:
def z(a,b):
    return a/b *100

z(0.388,0.333)

116.5165165165165